**USER-USER CF**

Zeyad Mohamed Fayed 
222102242

**Case Study 1**

In [30]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ----------------------------
# 1. Load dataset (Kaggle)
# ----------------------------
DATASET_PATH = "/kaggle/input/digital-music-csv/Digital_Music.csv"

ratings = pd.read_csv(
    DATASET_PATH,
    header=None,
    names=['user_id', 'product_id', 'rating', 'timestamp']
)

print("✅ Original dataset shape:", ratings.shape)

# ----------------------------
# 2. Take sample for faster processing
# ----------------------------
ratings_sample = ratings.sample(n=50000, random_state=42)
print("✅ Sampled dataset shape:", ratings_sample.shape)

# ----------------------------
# 3. Filter users/products with very few ratings
# ----------------------------
min_user_ratings = 5
min_item_ratings = 5

user_counts = ratings_sample['user_id'].value_counts()
item_counts = ratings_sample['product_id'].value_counts()

ratings_filtered = ratings_sample[
    (ratings_sample['user_id'].isin(user_counts[user_counts >= min_user_ratings].index)) &
    (ratings_sample['product_id'].isin(item_counts[item_counts >= min_item_ratings].index))
]

print("✅ Filtered dataset shape:", ratings_filtered.shape)

# ----------------------------
# 4. Create User-Item Matrix
# ----------------------------
user_item_matrix = ratings_filtered.pivot_table(
    index='user_id',
    columns='product_id',
    values='rating',
    aggfunc='mean'
).fillna(0)

print("✅ User-Item Matrix shape:", user_item_matrix.shape)

# ----------------------------
# 5. Drop users/items with all zeros (just in case)
# ----------------------------
user_item_matrix = user_item_matrix.loc[(user_item_matrix.sum(axis=1) > 0), 
                                        (user_item_matrix.sum(axis=0) > 0)]
print("✅ Cleaned User-Item Matrix shape:", user_item_matrix.shape)

# ----------------------------
# 6. Select target user (≥5 ratings)
# ----------------------------
# ----------------------------
# 6. Select target user (≥3 ratings) ✅
# ----------------------------
target_user_candidates = user_item_matrix[user_item_matrix.gt(0).sum(axis=1) >= 2].index

if len(target_user_candidates) == 0:
    raise ValueError("No target user found with >=3 ratings. Try lowering the threshold or increase sample size.")

target_user = target_user_candidates[0]
print(f"✅ Target user: {target_user} (rated {user_item_matrix.loc[target_user].gt(0).sum()} products)")


# 7. Filter users with overlap ✅
target_ratings = user_item_matrix.loc[target_user]

# خليه يجيب المستخدمين اللي عندهم على الأقل تقييم واحد مشترك مع target user
overlap_users_mask = user_item_matrix.apply(
    lambda row: ((row > 0) & (target_ratings > 0)).any(),
    axis=1
)

subset_users = user_item_matrix[overlap_users_mask]

print(f"✅ Users with overlap: {subset_users.shape[0]}")
print(f"✅ Subset User-Item Matrix shape: {subset_users.shape}")


# ----------------------------
# 8. Compute cosine similarity (filtered)
# ----------------------------
subset_similarity = cosine_similarity(subset_users, target_ratings.values.reshape(1, -1))
subset_similarity_df = pd.DataFrame(
    subset_similarity.flatten(),
    index=subset_users.index,
    columns=['similarity']
)

# استبعاد الـ target user نفسه
subset_similarity_df = subset_similarity_df.drop(index=target_user, errors='ignore')

# ----------------------------
# 9. Get Top 20% similar users
# ----------------------------
top_20_pct = max(int(len(subset_similarity_df) * 0.2), 1)
top_similar_users = subset_similarity_df.sort_values(by='similarity', ascending=False).head(top_20_pct)

print(f"\n✅ Top 20% Similar Users to User {target_user}:")
print(top_similar_users)


✅ Original dataset shape: (1584082, 4)
✅ Sampled dataset shape: (50000, 4)
✅ Filtered dataset shape: (105, 4)
✅ User-Item Matrix shape: (94, 64)
✅ Cleaned User-Item Matrix shape: (94, 64)
✅ Target user: B0012298LC (rated 2 products)
✅ Users with overlap: 2
✅ Subset User-Item Matrix shape: (2, 64)

✅ Top 20% Similar Users to User B0012298LC:
            similarity
user_id               
B00136LQC6        0.98


In [31]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ----------------------------
# 1. Load dataset
# ----------------------------
DATASET_PATH = "/kaggle/input/digital-music-csv/Digital_Music.csv"
ratings = pd.read_csv(
    DATASET_PATH,
    header=None,
    names=['user_id', 'product_id', 'rating', 'timestamp']
)

print(f"✅ Original dataset shape: {ratings.shape}")

# ----------------------------
# 2. Take a larger sample
# ----------------------------
sample_size = 1000000  # ممكن تزود لو عندك RAM
ratings_sample = ratings.sample(n=sample_size, random_state=42)
print(f"✅ Sampled dataset shape: {ratings_sample.shape}")

# ----------------------------
# 3. Filter users/products with very few ratings
# ----------------------------
min_user_ratings = 5
min_item_ratings = 5

user_counts = ratings_sample['user_id'].value_counts()
item_counts = ratings_sample['product_id'].value_counts()

ratings_filtered = ratings_sample[
    (ratings_sample['user_id'].isin(user_counts[user_counts >= min_user_ratings].index)) &
    (ratings_sample['product_id'].isin(item_counts[item_counts >= min_item_ratings].index))
]

print(f"✅ Filtered dataset shape: {ratings_filtered.shape}")

# ----------------------------
# 4. Create User-Item Matrix
# ----------------------------
user_item_matrix = ratings_filtered.pivot_table(
    index='user_id',
    columns='product_id',
    values='rating',
    aggfunc='mean'
).fillna(0)

# Drop all-zero users/items just in case
user_item_matrix = user_item_matrix.loc[(user_item_matrix.sum(axis=1) > 0),
                                        (user_item_matrix.sum(axis=0) > 0)]

print(f"✅ User-Item Matrix shape: {user_item_matrix.shape}")

# ----------------------------
# 5. Select target user (≥5 ratings)
# ----------------------------
target_user_candidates = user_item_matrix[user_item_matrix.gt(0).sum(axis=1) >= 5].index
if len(target_user_candidates) == 0:
    raise ValueError("No target user found with >=5 ratings. Try increasing sample size.")

target_user = target_user_candidates[0]
print(f"✅ Target user: {target_user} (rated {user_item_matrix.loc[target_user].gt(0).sum()} products)")

# ----------------------------
# 6. Filter users with any overlap (≥1 product in common)
# ----------------------------
rated_products = user_item_matrix.loc[target_user][user_item_matrix.loc[target_user] > 0].index
subset_users = user_item_matrix[user_item_matrix[rated_products].sum(axis=1) >= 1]
subset_users = subset_users.drop(index=target_user, errors='ignore')

print(f"✅ Users with overlap: {subset_users.shape[0]}")

# ----------------------------
# 7. Compute cosine similarity
# ----------------------------
subset_similarity = cosine_similarity(subset_users, user_item_matrix.loc[target_user].values.reshape(1, -1))
subset_similarity_df = pd.DataFrame(
    subset_similarity.flatten(),
    index=subset_users.index,
    columns=['similarity']
)

# ----------------------------
# 8. Select Top 20% similar users
# ----------------------------
top_20_pct = max(int(len(subset_similarity_df) * 0.4), 1)
top_similar_users = subset_similarity_df.sort_values(by='similarity', ascending=False).head(top_20_pct)
print(f"\n✅ Top 20% Similar Users:\n{top_similar_users}")

# ----------------------------
# 9. Predict unknown ratings
# ----------------------------
predictions = {}
global_means = user_item_matrix.mean(axis=0)  # fallback

for product in user_item_matrix.columns:
    if user_item_matrix.loc[target_user, product] == 0:  # unrated
        # ratings from top similar users
        ratings_by_similar = user_item_matrix.loc[top_similar_users.index, product]
        weights = top_similar_users['similarity']
        mask = ratings_by_similar > 0
        if mask.sum() > 0:
            # weighted average
            weighted_avg = np.dot(ratings_by_similar[mask], weights[mask]) / weights[mask].sum()
            predictions[product] = weighted_avg
        else:
            # fallback to global average
            predictions[product] = global_means[product]

# Convert to DataFrame
predicted_ratings_df = pd.DataFrame.from_dict(predictions, orient='index', columns=['predicted_rating'])
predicted_ratings_df = predicted_ratings_df.sort_values(by='predicted_rating', ascending=False)

print(f"\n✅ Predicted ratings for unrated products by {target_user}:")
print(predicted_ratings_df.head(10))


✅ Original dataset shape: (1584082, 4)
✅ Sampled dataset shape: (1000000, 4)
✅ Filtered dataset shape: (140283, 4)
✅ User-Item Matrix shape: (25938, 22740)
✅ Target user: 5557585400 (rated 6 products)
✅ Users with overlap: 19

✅ Top 20% Similar Users:
            similarity
user_id               
B007J7MQ54        0.41
B0080OK7YC        0.27
B008ZVVHXU        0.22
B00BCEQZMU        0.20
B001I8YKMC        0.20
B0018DS15O        0.19
B003ALAPLU        0.19

✅ Predicted ratings for unrated products by 5557585400:
                predicted_rating
AZWU085F8CUW2               5.00
A3GHWUGB9FG2PW              5.00
AJMYOAWK2XKS0               5.00
A1RWKI4KCX3K72              5.00
A2OGNFT0OGTBTH              5.00
A1UZTY06QS5X1Q              5.00
A22COT2LBTGICF              5.00
ATLZNVLYKP9AZ               5.00
A1A1W8YFZHVHC3              5.00
A3KXTPV7NIUUKK              5.00


In [32]:
# ----------------------------
# 1. Target user & top users
# ----------------------------
target_ratings = user_item_matrix.loc[target_user]
rated_products = target_ratings[target_ratings > 0].index

# Subset of users who rated at least 1 product in common
subset_users = user_item_matrix[user_item_matrix[rated_products].sum(axis=1) >= 1]
subset_users = subset_users.drop(index=target_user, errors='ignore')

# ----------------------------
# 2. Compute raw cosine similarity
# ----------------------------
raw_similarity = cosine_similarity(subset_users, target_ratings.values.reshape(1, -1)).flatten()
similarity_df = pd.DataFrame({
    'similarity': raw_similarity,
    'common_products': (subset_users[rated_products] > 0).sum(axis=1)
}, index=subset_users.index)

# ----------------------------
# 3. Compute Discount Factor (DF)
# ----------------------------
similarity_df['DF'] = similarity_df['common_products'] / len(rated_products)

# ----------------------------
# 4. Apply threshold ẞ >= 0.3
# ----------------------------
similarity_df = similarity_df[similarity_df['DF'] >= 0.1]

# ----------------------------
# 5. Compute Discounted Similarity (DS)
# ----------------------------
similarity_df['DS'] = similarity_df['similarity'] * similarity_df['DF']

# ----------------------------
# 6. Sort by DS
# ----------------------------
similarity_df = similarity_df.sort_values(by='DS', ascending=False)

print(f"\n✅ Discounted Similarities for User {target_user}:")
print(similarity_df.head(10))



✅ Discounted Similarities for User 5557585400:
            similarity  common_products   DF   DS
user_id                                          
B007J7MQ54        0.41                1 0.17 0.07
B0080OK7YC        0.27                1 0.17 0.04
B008ZVVHXU        0.22                1 0.17 0.04
B00BCEQZMU        0.20                1 0.17 0.03
B001I8YKMC        0.20                1 0.17 0.03
B0018DS15O        0.19                1 0.17 0.03
B003ALAPLU        0.19                1 0.17 0.03
B005G9D5G6        0.17                1 0.17 0.03
B000TER7I0        0.17                1 0.17 0.03
B0013JWOZG        0.16                1 0.17 0.03


In [33]:
# ----------------------------
# 1. Compute Top 20% users based on DS
# ----------------------------
top_20_pct_count = max(int(len(similarity_df) * 0.2), 1)  # تأكد إن على الأقل واحد
top_users_DS = similarity_df.sort_values(by='DS', ascending=False).head(top_20_pct_count)

print(f"\n✅ Top 20% Most Similar Users to {target_user} (using DS):")
print(top_users_DS)



✅ Top 20% Most Similar Users to 5557585400 (using DS):
            similarity  common_products   DF   DS
user_id                                          
B007J7MQ54        0.41                1 0.17 0.07
B0080OK7YC        0.27                1 0.17 0.04
B008ZVVHXU        0.22                1 0.17 0.04


In [34]:
# ----------------------------
# 1. Select target user's unrated products
# ----------------------------
target_ratings = user_item_matrix.loc[target_user]
unrated_products = target_ratings[target_ratings == 0].index

# ----------------------------
# 2. Predict ratings using Top DS users
# ----------------------------
predictions = {}
for product in unrated_products:
    # Get ratings of top DS users for this product
    ratings_by_top_users = user_item_matrix.loc[top_users_DS.index, product]
    weights = top_users_DS['DS']
    
    mask = ratings_by_top_users > 0  # only consider non-zero ratings
    if mask.sum() > 0:
        weighted_avg = np.dot(ratings_by_top_users[mask], weights[mask]) / weights[mask].sum()
        predictions[product] = weighted_avg
    else:
        # fallback: use average rating of the product from all users
        product_avg = user_item_matrix[product][user_item_matrix[product] > 0].mean()
        if not np.isnan(product_avg):
            predictions[product] = product_avg

# ----------------------------
# 3. Convert to DataFrame and sort
# ----------------------------
predicted_ratings_df = pd.DataFrame.from_dict(predictions, orient='index', columns=['predicted_rating'])
predicted_ratings_df = predicted_ratings_df.sort_values(by='predicted_rating', ascending=False)

print(f"\n✅ Predicted ratings for unrated products by {target_user}:")
print(predicted_ratings_df.head(10))



✅ Predicted ratings for unrated products by 5557585400:
                      predicted_rating
AZZI8LIBZYYIK                     5.00
AZYL2RTHUWR0P                     5.00
AZY0M1ANDSEPL                     5.00
A0108129TLIKAX34M8AA              5.00
A01241534EPLP5O3KOP5              5.00
A0234545X30ULJHGZUA3              5.00
A0698182HBSA2D2MEX40              5.00
A1006TXWG76H0N                    5.00
A1008539FMIMQDV67RVY              5.00
A100F1KITTQKKI                    5.00


**Case Study 2**

In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------
# 1. Load dataset & sample
# ----------------------------
DATASET_PATH = "/kaggle/input/digital-music-csv/Digital_Music.csv"
ratings = pd.read_csv(
    DATASET_PATH,
    header=None,
    names=['user_id', 'product_id', 'rating', 'timestamp']
)

# Very small sample to allow variety
ratings_sample = ratings.sample(n=20000, random_state=42)

# Filter users/products with few ratings
min_user_ratings = 1
min_item_ratings = 1
user_counts = ratings_sample['user_id'].value_counts()
item_counts = ratings_sample['product_id'].value_counts()

ratings_filtered = ratings_sample[
    (ratings_sample['user_id'].isin(user_counts[user_counts >= min_user_ratings].index)) &
    (ratings_sample['product_id'].isin(item_counts[item_counts >= min_item_ratings].index))
]

# ----------------------------
# 2. Create User-Item Matrix
# ----------------------------
user_item_matrix = ratings_filtered.pivot_table(
    index='user_id',
    columns='product_id',
    values='rating',
    aggfunc='mean'
).fillna(0)

user_item_matrix = user_item_matrix.loc[(user_item_matrix.sum(axis=1) > 0),
                                        (user_item_matrix.sum(axis=0) > 0)]

# ----------------------------
# 3. Select target user (1-2 ratings)
# ----------------------------
target_user_candidates = user_item_matrix[user_item_matrix.gt(0).sum(axis=1).between(1,2)].index
if len(target_user_candidates) == 0:
    raise ValueError("No user with 1-2 ratings. حاول sample أكبر أو خفف min_user_ratings.")
target_user = target_user_candidates[0]
print(f"✅ Target user: {target_user} (rated {user_item_matrix.loc[target_user].gt(0).sum()} products)")

# ----------------------------
# 4. Mean-center ratings
# ----------------------------
user_means = user_item_matrix.replace(0, np.NaN).mean(axis=1)
user_item_centered = user_item_matrix.sub(user_means, axis=0).fillna(0)

# ----------------------------
# 5. Filter users with any overlap
# ----------------------------
target_ratings_centered = user_item_centered.loc[target_user]
rated_products = target_ratings_centered[target_ratings_centered != 0].index

subset_users = user_item_centered[user_item_centered[rated_products].abs().sum(axis=1) > 0]
subset_users = subset_users.drop(index=target_user, errors='ignore')
print(f"✅ Users with overlap: {subset_users.shape[0]}")

# ----------------------------
# 6. Compute Mean-Centered Cosine similarity
# ----------------------------
similarities = cosine_similarity(subset_users, target_ratings_centered.values.reshape(1, -1)).flatten()
similarity_df = pd.DataFrame({
    'similarity': similarities,
    'common_products': (subset_users[rated_products] != 0).sum(axis=1)
}, index=subset_users.index)

# ----------------------------
# 7. Discount Factor & DS
# ----------------------------
similarity_df['DF'] = similarity_df['common_products'] / len(rated_products)
beta_threshold = 0.05  # very low to include variety
similarity_df = similarity_df[similarity_df['DF'] >= beta_threshold]
similarity_df['DS'] = similarity_df['similarity'] * similarity_df['DF']

similarity_df = similarity_df.sort_values(by='DS', ascending=False)
print(f"\n✅ Top DS Users (Mean-Centered) for {target_user}:")
print(similarity_df.head(20))

# ----------------------------
# 8. Users with DS < 1
# ----------------------------
variety_users = similarity_df[similarity_df['DS'] < 1].sort_values(by='DS', ascending=False)
print(f"\n✅ Users with DS < 1 for {target_user}:")
print(variety_users.head(10))


✅ Target user: 0001377647 (rated 1 products)
✅ Users with overlap: 15807

✅ Top DS Users (Mean-Centered) for 0001377647:
            similarity  common_products   DF   DS
user_id                                          
B001LM2ATA        1.00            19090 1.00 1.00
B00DUZDMBG        1.00            19090 1.00 1.00
B008BNARH4        1.00            19090 1.00 1.00
B01731JB9O        1.00            19090 1.00 1.00
B01BSA3V42        1.00            19090 1.00 1.00
B0042U8T6G        1.00            19090 1.00 1.00
B000Z7S5J6        1.00            19090 1.00 1.00
B008B57V1M        1.00            19090 1.00 1.00
B014M2BF2I        1.00            19090 1.00 1.00
B00137QUTO        1.00            19090 1.00 1.00
B000VZY9Z6        1.00            19090 1.00 1.00
B004RSCT3W        1.00            19090 1.00 1.00
B00JGEU1AU        1.00            19090 1.00 1.00
B00AIDIQ3C        1.00            19090 1.00 1.00
B000YNNTWY        1.00            19090 1.00 1.00
B00433B2H0        1.00       

In [36]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------
# 1. Create synthetic dataset
# ----------------------------
np.random.seed(42)

users = ['U1','U2','U3','U4','U5']
products = ['P1','P2','P3','P4','P5','P6','P7','P8']

# Random ratings 1-5
data = np.random.randint(1,6, size=(len(users), len(products)))

# Force some zeros for variety
data[0, 3:] = 0   # target user has rated only first 3 products
data[1, 0:2] = 0  # user with partial overlap
data[2, 3:] = 0   # another partial overlap
data[3, :] = 5    # fully rated
data[4, 2:5] = 0  # partial overlap

user_item_matrix = pd.DataFrame(data, index=users, columns=products)
print("✅ Synthetic User-Item Matrix:")
print(user_item_matrix)

# ----------------------------
# 2. Mean-center
# ----------------------------
user_means = user_item_matrix.replace(0, np.NaN).mean(axis=1)
user_item_centered = user_item_matrix.sub(user_means, axis=0).fillna(0)

# ----------------------------
# 3. Target user
# ----------------------------
target_user = 'U1'
target_ratings = user_item_centered.loc[target_user]
rated_products = target_ratings[target_ratings != 0].index

# ----------------------------
# 4. Users with overlap
# ----------------------------
subset_users = user_item_centered[user_item_centered[rated_products].abs().sum(axis=1) > 0]
subset_users = subset_users.drop(index=target_user, errors='ignore')

# ----------------------------
# 5. Cosine similarity
# ----------------------------
similarities = cosine_similarity(subset_users, target_ratings.values.reshape(1,-1)).flatten()
similarity_df = pd.DataFrame({
    'similarity': similarities,
    'common_products': (subset_users[rated_products] != 0).sum(axis=1)
}, index=subset_users.index)

# ----------------------------
# 6. Discount Factor (DF) and DS
# ----------------------------
similarity_df['DF'] = similarity_df['common_products'] / len(rated_products)
similarity_df['DS'] = similarity_df['similarity'] * similarity_df['DF']

# Sort by DS
similarity_df = similarity_df.sort_values(by='DS', ascending=False)
print("\n✅ DS values for target user:")
print(similarity_df)

# ----------------------------
# 7. Show variety: DS=1 and DS<1
variety_users = similarity_df[similarity_df['DS'] < 1]
print("\n✅ Users with DS < 1:")
print(variety_users)


✅ Synthetic User-Item Matrix:
    P1  P2  P3  P4  P5  P6  P7  P8
U1   4   5   3   0   0   0   0   0
U2   0   0   4   3   5   2   4   2
U3   4   5   1   0   0   0   0   0
U4   5   5   5   5   5   5   5   5
U5   4   1   0   0   0   5   1   2

✅ DS values for target user:
    similarity  common_products   DF    DS
U3        0.97                7 1.00  0.97
U5        0.40                7 1.00  0.40
U2       -0.03                7 1.00 -0.03

✅ Users with DS < 1:
    similarity  common_products   DF    DS
U3        0.97                7 1.00  0.97
U5        0.40                7 1.00  0.40
U2       -0.03                7 1.00 -0.03


In [37]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------
# 1. Create larger synthetic dataset
# ----------------------------
np.random.seed(42)

users = [f'U{i}' for i in range(1, 11)]       # 10 users
products = [f'P{j}' for j in range(1, 13)]    # 12 products

# Random ratings 1-5
data = np.random.randint(1, 6, size=(len(users), len(products)))

# Force some zeros for variety
data[0, 6:] = 0   # target user rated only first 6 products
data[1, 0:3] = 0
data[2, 4:] = 0
data[3, :] = 5    # fully rated, high similarity
data[4, 2:6] = 0
data[5, 0:2] = 0
data[6, 5:] = 0
data[7, :] = 3    # fully rated but lower than 5
data[8, 1:4] = 0
data[9, 0:1] = 0

user_item_matrix = pd.DataFrame(data, index=users, columns=products)
print("✅ Synthetic User-Item Matrix:")
print(user_item_matrix)

# ----------------------------
# 2. Mean-center
# ----------------------------
user_means = user_item_matrix.replace(0, np.NaN).mean(axis=1)
user_item_centered = user_item_matrix.sub(user_means, axis=0).fillna(0)

# ----------------------------
# 3. Target user
# ----------------------------
target_user = 'U1'
target_ratings = user_item_centered.loc[target_user]
rated_products = target_ratings[target_ratings != 0].index

# ----------------------------
# 4. Users with overlap
# ----------------------------
subset_users = user_item_centered[user_item_centered[rated_products].abs().sum(axis=1) > 0]
subset_users = subset_users.drop(index=target_user, errors='ignore')

# ----------------------------
# 5. Compute Cosine similarity
# ----------------------------
similarities = cosine_similarity(subset_users, target_ratings.values.reshape(1,-1)).flatten()
similarity_df = pd.DataFrame({
    'similarity': similarities,
    'common_products': (subset_users[rated_products] != 0).sum(axis=1)
}, index=subset_users.index)

# ----------------------------
# 6. Discount Factor (DF) and DS
# ----------------------------
similarity_df['DF'] = similarity_df['common_products'] / len(rated_products)
similarity_df['DS'] = similarity_df['similarity'] * similarity_df['DF']

# Sort by DS
similarity_df = similarity_df.sort_values(by='DS', ascending=False)
print("\n✅ DS values for target user:")
print(similarity_df)

# ----------------------------
# 7. Top 20% DS Users
top_20_percent_count = max(1, int(0.2 * len(similarity_df)))  # at least 1 user
top_users_20pct = similarity_df.head(top_20_percent_count)

print(f"\n✅ Top 20% DS Users for {target_user}:")
print(top_users_20pct)

# ----------------------------
# 8. Predict ratings for unrated products
predictions_top20 = {}
top_users = top_users_20pct.index
weights = top_users_20pct['DS']

for product in user_item_matrix.columns:
    if user_item_matrix.loc[target_user, product] == 0:  # unrated
        ratings_by_top = user_item_matrix.loc[top_users, product]
        mask = ratings_by_top > 0
        if mask.sum() > 0:
            weighted_avg = np.dot(ratings_by_top[mask], weights[mask]) / weights[mask].sum()
            predictions_top20[product] = weighted_avg

predicted_ratings_top20_df = pd.DataFrame.from_dict(predictions_top20, orient='index', columns=['predicted_rating'])
predicted_ratings_top20_df = predicted_ratings_top20_df.sort_values(by='predicted_rating', ascending=False)

print(f"\n✅ Predicted ratings for unrated products by {target_user} (Top 20% DS Users):")
print(predicted_ratings_top20_df)


✅ Synthetic User-Item Matrix:
     P1  P2  P3  P4  P5  P6  P7  P8  P9  P10  P11  P12
U1    4   5   3   5   5   2   0   0   0    0    0    0
U2    0   0   0   2   4   5   1   4   2    5    4    1
U3    1   3   3   2   0   0   0   0   0    0    0    0
U4    5   5   5   5   5   5   5   5   5    5    5    5
U5    2   4   0   0   0   0   3   1   4    2    4    2
U6    0   0   5   2   2   4   2   2   4    4    1    5
U7    5   2   5   2   1   0   0   0   0    0    0    0
U8    3   3   3   3   3   3   3   3   3    3    3    3
U9    5   0   0   0   4   1   4   2   1    5    3    4
U10   0   3   1   3   5   3   1   5   2    3    1    2

✅ DS values for target user:
     similarity  common_products   DF    DS
U3         0.83               11 1.00  0.83
U7         0.80               11 1.00  0.80
U10        0.21               11 1.00  0.21
U5         0.09               11 1.00  0.09
U9         0.05               11 1.00  0.05
U2         0.04               11 1.00  0.04
U6        -0.10            

In [38]:
top_20_percent_count = max(1, int(0.2 * len(similarity_df)))  # at least 1 user
top_users_20pct = similarity_df.head(top_20_percent_count)

print(f"\n✅ Top 20% DS Users for {target_user}:")
print(top_users_20pct)


✅ Top 20% DS Users for U1:
    similarity  common_products   DF   DS
U3        0.83               11 1.00 0.83


In [39]:
# ----------------------------
# 8. Predict ratings for unrated products using top 20% DS users
# ----------------------------
predictions_top20 = {}
top_users = top_users_20pct.index
weights = top_users_20pct['DS']

for product in user_item_matrix.columns:
    if user_item_matrix.loc[target_user, product] == 0:  # unrated
        ratings_by_top = user_item_matrix.loc[top_users, product]
        mask = ratings_by_top > 0
        if mask.sum() > 0:
            weighted_avg = np.dot(ratings_by_top[mask], weights[mask]) / weights[mask].sum()
            predictions_top20[product] = weighted_avg

predicted_ratings_top20_df = pd.DataFrame.from_dict(predictions_top20, orient='index', columns=['predicted_rating'])
predicted_ratings_top20_df = predicted_ratings_top20_df.sort_values(by='predicted_rating', ascending=False)

print(f"\n✅ Predicted ratings for unrated products by {target_user} (Top 20% DS Users):")
print(predicted_ratings_top20_df)



✅ Predicted ratings for unrated products by U1 (Top 20% DS Users):
Empty DataFrame
Columns: [predicted_rating]
Index: []


In [40]:
# ----------------------------
# 1. Predict ratings for unrated products using top 20% DS users
# ----------------------------
predictions_top20 = {}
top_users = top_users_20pct.index
weights = top_users_20pct['DS']

for product in user_item_matrix.columns:
    if user_item_matrix.loc[target_user, product] == 0:
        ratings_by_top = user_item_matrix.loc[top_users, product]
        mask = ratings_by_top > 0
        if mask.sum() > 0:
            weighted_avg = np.dot(ratings_by_top[mask], weights[mask]) / weights[mask].sum()
        else:
            weighted_avg = user_item_matrix[product][user_item_matrix[product]>0].mean()  # fallback
        predictions_top20[product] = weighted_avg


# تحويل النتائج إلى DataFrame مرتبة
predicted_ratings_top20_df = pd.DataFrame.from_dict(predictions_top20, orient='index', columns=['predicted_rating'])
predicted_ratings_top20_df = predicted_ratings_top20_df.sort_values(by='predicted_rating', ascending=False)

print(f"\n✅ Predicted ratings for unrated products by {target_user} (Top 20% DS Users):")
print(predicted_ratings_top20_df)



✅ Predicted ratings for unrated products by U1 (Top 20% DS Users):
     predicted_rating
P10              3.86
P8               3.14
P12              3.14
P9               3.00
P11              3.00
P7               2.71


In [41]:
# β threshold
beta_threshold = 0.3  # يمكن تغييره حسب الحاجة

# حساب DF
similarity_df['DF'] = similarity_df['common_products'] / len(rated_products)

# الاحتفاظ فقط بالمستخدمين الذين DF >= β
similarity_df = similarity_df[similarity_df['DF'] >= beta_threshold]

# حساب DS
similarity_df['DS'] = similarity_df['similarity'] * similarity_df['DF']

# ترتيب المستخدمين حسب DS
similarity_df = similarity_df.sort_values(by='DS', ascending=False)

print("\n✅ DF and DS values after applying β threshold:")
print(similarity_df)



✅ DF and DS values after applying β threshold:
     similarity  common_products   DF    DS
U3         0.83               11 1.00  0.83
U7         0.80               11 1.00  0.80
U10        0.21               11 1.00  0.21
U5         0.09               11 1.00  0.09
U9         0.05               11 1.00  0.05
U2         0.04               11 1.00  0.04
U6        -0.10               11 1.00 -0.10


In [42]:
# حساب عدد المستخدمين في أعلى 20%
top_20_percent_count = max(1, int(0.2 * len(similarity_df)))  # على الأقل 1 user

# اختيار أعلى المستخدمين حسب DS
top_users_20pct = similarity_df.head(top_20_percent_count)

print(f"\n✅ Top 20% DS Users for target user:")
print(top_users_20pct)



✅ Top 20% DS Users for target user:
    similarity  common_products   DF   DS
U3        0.83               11 1.00 0.83


In [43]:
# ----------------------------
# Predict ratings for unrated products using top DS users with fallback
# ----------------------------
predictions_top20 = {}
top_users = top_users_20pct.index
weights = top_users_20pct['DS']

for product in user_item_matrix.columns:
    if user_item_matrix.loc[target_user, product] == 0:  # unrated
        ratings_by_top = user_item_matrix.loc[top_users, product]
        mask = ratings_by_top > 0
        if mask.sum() > 0:
            # Weighted average باستخدام DS
            weighted_avg = np.dot(ratings_by_top[mask], weights[mask]) / weights[mask].sum()
        else:
            # Fallback: متوسط تقييم المنتج عالميًا
            weighted_avg = user_item_matrix[product][user_item_matrix[product] > 0].mean()
        predictions_top20[product] = weighted_avg

# تحويل النتائج إلى DataFrame مرتبة
predicted_ratings_top20_df = pd.DataFrame.from_dict(
    predictions_top20, orient='index', columns=['predicted_rating']
)
predicted_ratings_top20_df = predicted_ratings_top20_df.sort_values(
    by='predicted_rating', ascending=False
)

print(f"\n✅ Predicted ratings for unrated products by {target_user} (Top DS Users with fallback):")
print(predicted_ratings_top20_df.head(10))



✅ Predicted ratings for unrated products by U1 (Top DS Users with fallback):
     predicted_rating
P10              3.86
P8               3.14
P12              3.14
P9               3.00
P11              3.00
P7               2.71


**Case Study 3**

In [63]:
import pandas as pd
import numpy as np

# 1. Load Data
DATASET_PATH = "/kaggle/input/digital-music-csv/Digital_Music.csv"
ratings = pd.read_csv(DATASET_PATH, header=None, names=['user_id', 'product_id', 'rating', 'timestamp'])

# ---------------------------------------------------------
# خطوة 1: تنظيف ذكي (نأخذ فقط من قيموا أكثر من 10 منتجات)
# لنحصل على مستخدمين "حقيقيين" وليس عشوائيين
# ---------------------------------------------------------
user_counts = ratings['user_id'].value_counts()
active_users = user_counts[user_counts >= 10].index
ratings_filtered = ratings[ratings['user_id'].isin(active_users)]

# نأخذ عينة من هؤلاء النشطين فقط
ratings_sample = ratings_filtered.head(100000) 

# ---------------------------------------------------------
# خطوة 2: بناء المصفوفة
# ---------------------------------------------------------
user_item_matrix = ratings_sample.pivot_table(
    index='user_id', 
    columns='product_id', 
    values='rating'
) # لا تضع fillna(0) هنا لأننا نحتاج NaN لحساب بيرسون الصحيح

# اختيار هدف لديه تقييمات كثيرة
target_user = user_item_matrix.count(axis=1).idxmax()
print(f"Target User: {target_user} (Rated {user_item_matrix.loc[target_user].count()} items)")

# ---------------------------------------------------------
# خطوة 3: حساب PCC مع "عقاب" للعدد القليل (Shrinkage)
# ---------------------------------------------------------
target_ratings = user_item_matrix.loc[target_user]
pcc_dict = {}

# عامل العقاب: كلما زاد الرقم، عاقبنا التداخل القليل أكثر
SHRINKAGE_PARAMETER = 5  

for user in user_item_matrix.index:
    if user == target_user:
        continue
    
    other_ratings = user_item_matrix.loc[user]
    
    # نأخذ فقط المنتجات التي قيمها الاثنان (ليست NaN عند الطرفين)
    common = pd.concat([target_ratings, other_ratings], axis=1, join='inner').dropna()
    n_common = len(common)
    
    # الشرط: يجب أن يشتركوا في 3 منتجات على الأقل
    if n_common < 3:
        continue

    x = common.iloc[:, 0].values
    y = common.iloc[:, 1].values

    # إذا كانت التقييمات ثابتة (مثلاً كلها 5)، الانحراف المعياري 0 ولا يمكن حساب الارتباط
    if np.std(x) == 0 or np.std(y) == 0:
        continue

    # حساب بيرسون الطبيعي
    corr = np.corrcoef(x, y)[0, 1]
    
    # 🔥 هنا الحل السحري: تقليل القيمة بناء على عدد المشتركات
    # لو اشتركوا في 3 منتجات فقط: المعامل سيضرب في (3 / 8) = 0.37 (يقلل التشابه جداً)
    # لو اشتركوا في 50 منتج: المعامل سيضرب في (50 / 55) = 0.90 (يحافظ على التشابه)
    weighted_corr = corr * (n_common / (n_common + SHRINKAGE_PARAMETER))
    
    pcc_dict[user] = {
        'Sim': weighted_corr, 
        'Raw_Corr': corr, 
        'Common_Items': n_common
    }

# تحويل للنتائج
results = pd.DataFrame.from_dict(pcc_dict, orient='index')

if not results.empty:
    results = results.sort_values(by='Sim', ascending=False)
    print("\n✅ Top Similar Users (Weighted PCC):")
    print(results.head(10))
else:
    print("لا يوجد مستخدمين حققوا الشروط (جرب تقليل شرط n_common أو زيادة العينة)")

Target User: B000V6ADNW (Rated 1058 items)

✅ Top Similar Users (Weighted PCC):
             Sim  Raw_Corr  Common_Items
B000VRTJI6  0.56      0.95             7
B0014JIVWA  0.22      0.49             4
B000W11N6W  0.11      0.21             5
B00122909W  0.09      0.15             8
B001237HCI -0.12     -0.25             5
B0011YBRFQ -0.19     -0.50             3
B00122KC4E -0.26     -0.58             4
B000W0TIRE -0.29     -0.66             4
9714721180 -0.37     -0.46            21
B000WKT6B2 -0.37     -1.00             3


In [64]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------
# 1. Load & Filter Data (Smart Sampling)
# ----------------------------
DATASET_PATH = "/kaggle/input/digital-music-csv/Digital_Music.csv"
ratings = pd.read_csv(DATASET_PATH, header=None, names=['user_id', 'product_id', 'rating', 'timestamp'])

# تصفية للحصول على بيانات كثيفة (Dense) لتجنب الأصفار
# نأخذ المستخدمين الذين قيموا 10 منتجات على الأقل
user_counts = ratings['user_id'].value_counts()
active_users = user_counts[user_counts >= 10].index
ratings_filtered = ratings[ratings['user_id'].isin(active_users)]

# نأخذ المنتجات التي تم تقييمها 10 مرات على الأقل
item_counts = ratings_filtered['product_id'].value_counts()
popular_items = item_counts[item_counts >= 10].index
ratings_final = ratings_filtered[ratings_filtered['product_id'].isin(popular_items)]

# نأخذ عينة (مثلاً أول 1000 مستخدم نشط لسرعة المعالجة)
# يمكنك زيادة الرقم حسب قوة جهازك
unique_users = ratings_final['user_id'].unique()[:1000] 
ratings_final = ratings_final[ratings_final['user_id'].isin(unique_users)]

print(f"✅ Data Shape: {ratings_final.shape} | Users: {len(unique_users)}")

# ----------------------------
# 2. Create User-Item Matrix
# ----------------------------
user_item_matrix = ratings_final.pivot_table(
    index='user_id', 
    columns='product_id', 
    values='rating'
).fillna(0)

# ----------------------------
# 3. Mean-Centered Cosine Similarity (Vectorized Pearson)
# ----------------------------
# طرح المتوسط من كل صف (مع تجاهل الأصفار في الحساب الأصلي)
# هنا سنطرح المتوسط ببساطة (تقريب مقبول للسرعة)
user_means = user_item_matrix.mean(axis=1)
matrix_centered = user_item_matrix.sub(user_means, axis=0)

# حساب التشابه بين الجميع (N x N Matrix)
sim_matrix = cosine_similarity(matrix_centered)

# تحويلها لـ DataFrame ليسهل التعامل مع الأسماء
sim_df = pd.DataFrame(sim_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

# ----------------------------
# 4. Get Top 20% Neighbors for EACH User
# ----------------------------
top_20_percent_dict = {}

# عدد الجيران الكلي (نطرح 1 لأن المستخدم لا يحسب مع نفسه)
total_neighbors = len(sim_df) - 1
k_top = int(total_neighbors * 0.20)  # حساب عدد الـ 20%

print(f"🔍 Finding top {k_top} neighbors (20%) for each user...\n")

for user in sim_df.index:
    # 1. جلب صف المستخدم
    user_similarities = sim_df.loc[user]
    
    # 2. حذف المستخدم نفسه من المقارنة (لأن التشابه 1.0)
    user_similarities = user_similarities.drop(user)
    
    # 3. الترتيب تنازلياً (الأعلى تشابهاً أولاً)
    sorted_sim = user_similarities.sort_values(ascending=False)
    
    # 4. أخذ أول 20%
    top_neighbors = sorted_sim.head(k_top)
    
    # التخزين
    top_20_percent_dict[user] = top_neighbors.index.tolist()

# ----------------------------
# 5. Show Results Example
# ----------------------------
# عرض النتائج لأول 3 مستخدمين فقط كعينة
for i, (user, neighbors) in enumerate(top_20_percent_dict.items()):
    if i >= 3: break
    print(f"👤 User: {user}")
    print(f"   -> Top 20% Neighbors ({len(neighbors)} users): {neighbors[:5]} ...")
    print("-" * 50)

# إذا أردت حفظ النتائج في ملف
# output_df = pd.DataFrame.from_dict(top_20_percent_dict, orient='index')
# output_df.to_csv("top_20_percent_neighbors.csv")

✅ Data Shape: (8412, 4) | Users: 1000
🔍 Finding top 199 neighbors (20%) for each user...

👤 User: 0001377647
   -> Top 20% Neighbors (199 users): ['B000TDWSYO', 'B000QVXDR0', 'B000VZO4TW', 'B000S56380', 'B000T1EJ0W'] ...
--------------------------------------------------
👤 User: 0001388703
   -> Top 20% Neighbors (199 users): ['B000VZJS84', '5557386823', 'B000V697Z2', 'B000GSACA4', 'B000TE3JTQ'] ...
--------------------------------------------------
👤 User: 0006920055
   -> Top 20% Neighbors (199 users): ['5557386823', 'B000TE3JTQ', '0001388703', 'B000GSACA4', 'B0009Z0SRG'] ...
--------------------------------------------------


In [65]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------
# 1. تحميل الملف الأصلي وتصفيته (لإدارة الذاكرة)
# ----------------------------
print("⏳ Loading Original Dataset...")
DATASET_PATH = "/kaggle/input/digital-music-csv/Digital_Music.csv"
ratings = pd.read_csv(
    DATASET_PATH, 
    header=None, 
    names=['user_id', 'product_id', 'rating', 'timestamp']
)

print(f"Original Data Size: {ratings.shape}")

# --- خطوة 1: تصفية البيانات لضمان الجودة وتقليل الحجم ---
# سنحتفظ فقط بالمستخدمين الذين قيموا 20 منتجاً على الأقل
# والمنتجات التي حصلت على 20 تقييماً على الأقل (لضمان التقاطع)
min_user_ratings = 20
min_item_ratings = 20

# تصفية المستخدمين
user_counts = ratings['user_id'].value_counts()
active_users = user_counts[user_counts >= min_user_ratings].index
ratings_filtered = ratings[ratings['user_id'].isin(active_users)]

# تصفية المنتجات (من البيانات المفلترة)
item_counts = ratings_filtered['product_id'].value_counts()
popular_items = item_counts[item_counts >= min_item_ratings].index
ratings_final = ratings_filtered[ratings_filtered['product_id'].isin(popular_items)]

# تحذير: إذا كانت البيانات لا تزال ضخمة جداً، نأخذ أعلى 1000 مستخدم نشاطاً فقط
if len(ratings_final['user_id'].unique()) > 2000:
    top_2000_users = ratings_final['user_id'].value_counts().head(2000).index
    ratings_final = ratings_final[ratings_final['user_id'].isin(top_2000_users)]

print(f"✅ Filtered Data Size: {ratings_final.shape}")
print(f"   Users: {ratings_final['user_id'].nunique()} | Items: {ratings_final['product_id'].nunique()}")

# ----------------------------
# 2. بناء المصفوفة (User-Item Matrix)
# ----------------------------
user_item_matrix = ratings_final.pivot_table(
    index='user_id', 
    columns='product_id', 
    values='rating'
).fillna(0)

# حساب المتوسط الحقيقي لكل مستخدم (بدون الأصفار)
true_user_means = user_item_matrix.replace(0, np.nan).mean(axis=1)

# ----------------------------
# 3. حساب التشابه وتحديد الجيران (Top 20%)
# ----------------------------
# Mean-Centered Matrix (User - Mean)
matrix_centered = user_item_matrix.sub(true_user_means, axis=0).fillna(0)

# Cosine Similarity (يكافئ Pearson بعد الطرح)
sim_matrix = cosine_similarity(matrix_centered)
sim_df = pd.DataFrame(sim_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

# تحديد أفضل 20% جيران لكل مستخدم
neighbors_dict = {}
k_percentage = 0.20 # نسبة 20%
total_candidates = len(sim_df) - 1 # الكل ما عدا المستخدم نفسه
k_neighbors = max(1, int(total_candidates * k_percentage))

print(f"🔍 Finding top {k_neighbors} neighbors (20%) per user...")

for user in sim_df.index:
    # ترتيب التنازلي واستبعاد المستخدم نفسه
    user_sims = sim_df.loc[user].drop(user).sort_values(ascending=False)
    # أخذ النسبة المحددة
    top_neighbors = user_sims.head(k_neighbors).index.tolist()
    neighbors_dict[user] = top_neighbors

# ----------------------------
# 4. التنبؤ بالتقييمات المفقودة (Rating Prediction)
# ----------------------------
predictions = []

print("🚀 Starting Prediction Loop (Active Users Only)...")

# مصفوفة الفروقات (Rating - Mean) لاستخدامها في الحسابات بسرعة
ratings_diff_matrix = user_item_matrix.replace(0, np.nan).sub(true_user_means, axis=0)

# للتسريع: سنقوم بالتنبؤ لأول 10 مستخدمين فقط كعينة (لأن التنبؤ للكل يستغرق وقتاً طويلاً)
# يمكنك إزالة [:10] لحساب الكل
target_users_sample = user_item_matrix.index[:10] 

for target_user in target_users_sample:
    
    # 1. تحديد الجيران
    my_neighbors = neighbors_dict[target_user]
    
    # 2. ما هي المنتجات التي لم يقيمها المستخدم؟ (قيمتها 0)
    user_row = user_item_matrix.loc[target_user]
    unrated_items = user_row[user_row == 0].index
    
    # لا نريد التنبؤ بمليون منتج، نختار فقط المنتجات التي قيمها الجيران
    # نأخذ تقييمات الجيران للمنتجات غير المقيمة
    neighbors_ratings = user_item_matrix.loc[my_neighbors, unrated_items]
    
    # نأخذ فقط المنتجات التي يوجد لها تقييم واحد على الأقل من الجيران
    items_to_predict = neighbors_ratings.columns[neighbors_ratings.sum() > 0]
    
    for item in items_to_predict:
        # نحتاج الجيران الذين قيموا هذا المنتج فعلياً
        # (Ratings > 0)
        raters = neighbors_ratings.index[neighbors_ratings[item] > 0]
        
        if len(raters) == 0:
            continue
            
        # المعادلة:
        # Prediction = Mean_u + (Sum(Sim * (R_v - Mean_v)) / Sum(|Sim|))
        
        # البسط: التشابه * (تقييم الجار - متوسط الجار)
        sim_scores = sim_df.loc[target_user, raters]
        neighbor_diffs = ratings_diff_matrix.loc[raters, item] # القيمة جاهزة
        
        numerator = np.dot(sim_scores, neighbor_diffs)
        denominator = sim_scores.abs().sum()
        
        if denominator == 0:
            continue
            
        predicted_rating = true_user_means[target_user] + (numerator / denominator)
        
        # ضبط النتيجة بين 1 و 5
        predicted_rating = np.clip(predicted_rating, 1, 5)
        
        predictions.append({
            'User': target_user,
            'Product': item,
            'Predicted_Rating': round(predicted_rating, 2)
        })

# ----------------------------
# 5. عرض النتائج
# ----------------------------
pred_df = pd.DataFrame(predictions)

if not pred_df.empty:
    print("\n✅ Top 10 Predicted Ratings (User-Based CF):")
    # ترتيب حسب أعلى تقييم متوقع
    print(pred_df.sort_values(by='Predicted_Rating', ascending=False).head(10))
    
    # إحصائيات
    print(f"\nTotal Predictions Made: {len(pred_df)}")
else:
    print("No predictions made (Not enough overlap in this sample).")

⏳ Loading Original Dataset...
Original Data Size: (1584082, 4)
✅ Filtered Data Size: (23565, 4)
   Users: 2000 | Items: 1050
🔍 Finding top 399 neighbors (20%) per user...
🚀 Starting Prediction Loop (Active Users Only)...

✅ Top 10 Predicted Ratings (User-Based CF):
            User        Product  Predicted_Rating
7032  B000S43HH6  AS0CCKP5ZB0M6              5.00
7033  B000S43HH6  ASDPTCEGEINUZ              5.00
7035  B000S43HH6  ASU3NPI7FLQBA              5.00
7036  B000S43HH6  ASVNSWIXBV72Q              5.00
7024  B000S43HH6  AQFW7GDBQ0DPE              5.00
7025  B000S43HH6  AQHVR2V3YA1MM              5.00
7026  B000S43HH6  AR3N1VLW722V9              5.00
7027  B000S43HH6  AR8O7SVSEUI9Z              5.00
7028  B000S43HH6  ARIXGO75TOE4K              5.00
7029  B000S43HH6  ARQ2J7D4DZTRL              5.00

Total Predictions Made: 7074


In [67]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1. Load & Filter Original Data
# ---------------------------------------------------------
# We use smart filtering to keep the code fast and memory-safe
print("⏳ Loading and Preprocessing...")

DATASET_PATH = "/kaggle/input/digital-music-csv/Digital_Music.csv"
ratings = pd.read_csv(DATASET_PATH, header=None, names=['user_id', 'product_id', 'rating', 'timestamp'])

# Filter: Keep users with > 10 ratings and items with > 10 ratings
# This ensures we actually have overlaps to calculate similarity
min_activity = 10
user_counts = ratings['user_id'].value_counts()
active_users = user_counts[user_counts >= min_activity].index
ratings = ratings[ratings['user_id'].isin(active_users)]

item_counts = ratings['product_id'].value_counts()
popular_items = item_counts[item_counts >= min_activity].index
ratings = ratings[ratings['product_id'].isin(popular_items)]

# For demonstration speed, let's take the top 1,000 active users
# (You can increase this if you have more RAM)
top_users = ratings['user_id'].value_counts().head(1000).index
ratings_final = ratings[ratings['user_id'].isin(top_users)]

print(f"✅ Data Ready: {ratings_final.shape} | Users: {ratings_final['user_id'].nunique()}")

# ---------------------------------------------------------
# 2. Create Matrix
# ---------------------------------------------------------
# Use float for NaN support
user_item_matrix = ratings_final.pivot_table(
    index='user_id', 
    columns='product_id', 
    values='rating'
)

# ---------------------------------------------------------
# 3. Define Discounted Similarity Function
# ---------------------------------------------------------
def calculate_discounted_similarity(matrix_df, beta=1.0, shrinkage=5):
    """
    Computes similarity based on:
    1. Threshold Beta: Ignore items where |diff| > beta
    2. Distance Penalty: Closer ratings = Higher score
    3. Discounting (Shrinkage): Penalize low common item counts
    """
    users = matrix_df.index
    n_users = len(users)
    sim_matrix = np.zeros((n_users, n_users))
    
    # Convert to numpy array for speed
    # Shape: (Users, Items)
    R = matrix_df.values
    
    print(f"🚀 Calculating Discounted Similarities (Beta={beta})...")
    
    # Loop over users (optimized inner operations)
    for i in range(n_users):
        # User u vector
        u_vec = R[i]
        
        # We compare User u against ALL other users at once (Vectorization)
        # diffs shape: (n_users, n_items)
        # abs(R - u_vec) calculates difference between User i and everyone else
        diffs = np.abs(R - u_vec)
        
        # Masks
        # 1. Valid ratings: Both u and v must have rated the item (not NaN)
        # (np.isnan check is needed because we didn't fillna(0))
        isnan_mask = np.isnan(diffs) 
        
        # 2. Threshold check: Difference must be <= Beta
        threshold_mask = diffs <= beta
        
        # Final mask: Valid ratings AND within threshold
        # items that are NaN will be False in threshold_mask anyway if strictly handled, 
        # but explicit check is safer.
        valid_mask = (~np.isnan(R)) & (~np.isnan(u_vec)) & threshold_mask
        
        # Count common items passing the threshold
        common_counts = np.sum(valid_mask, axis=1)
        
        # Calculate raw similarity sum for valid items
        # Score per item = 1 - (diff / (beta + 1))
        # If diff=0 -> score=1. If diff=1 (beta) -> score=0.5
        item_scores = 1 - (diffs / (beta + 1))
        
        # Zero out invalid items
        item_scores[~valid_mask] = 0
        
        # Sum of scores per user
        sum_scores = np.sum(item_scores, axis=1)
        
        # Average Similarity (Sum / Count)
        # Avoid division by zero
        avg_sim = np.zeros_like(sum_scores)
        nonzero_mask = common_counts > 0
        avg_sim[nonzero_mask] = sum_scores[nonzero_mask] / common_counts[nonzero_mask]
        
        # --- DISCOUNTING STEP ---
        # Apply Shrinkage: Sim * (n / (n + shrinkage))
        discount_factor = common_counts / (common_counts + shrinkage)
        final_sim = avg_sim * discount_factor
        
        sim_matrix[i] = final_sim
        
    # Set diagonal to 1 (or 0 if we want to ignore self)
    np.fill_diagonal(sim_matrix, 1.0)
    
    return pd.DataFrame(sim_matrix, index=users, columns=users)

# Run Calculation
# Beta=1: Tolerates a difference of 1 star.
sim_df = calculate_discounted_similarity(user_item_matrix, beta=1.0, shrinkage=3)

# ---------------------------------------------------------
# 4. Identify Top 20% Neighbors
# ---------------------------------------------------------
top_20_neighbors = {}
total_candidates = len(sim_df) - 1
k_top = max(1, int(total_candidates * 0.20)) # 20% count

print(f"\n🔍 Extracting top {k_top} neighbors (20%) per user...")

for user in sim_df.index:
    # 1. Get row, drop self
    user_sims = sim_df.loc[user].drop(user)
    
    # 2. Filter out users with 0 similarity (no valid overlap)
    user_sims = user_sims[user_sims > 0]
    
    # 3. Sort Descending
    user_sims = user_sims.sort_values(ascending=False)
    
    # 4. Slice top 20%
    # If valid neighbors are fewer than k_top, we take what we have
    current_k = min(len(user_sims), k_top)
    best_neighbors = user_sims.head(current_k).index.tolist()
    
    top_20_neighbors[user] = best_neighbors

# ---------------------------------------------------------
# 5. Display Results
# ---------------------------------------------------------
print("\n✅ Results (Discounted Distance-Based Similarity):")
count = 0
for user, neighbors in top_20_neighbors.items():
    if len(neighbors) > 0:
        print(f"User {user}:")
        # Print top 5 just to see scores
        top_5_scores = sim_df.loc[user, neighbors[:5]].values
        print(f"   -> Top Neighbors: {neighbors[:5]}...")
        print(f"   -> Scores: {np.round(top_5_scores, 3)}")
        count += 1
    if count >= 5: break

# Stats
neighbor_counts = [len(n) for n in top_20_neighbors.values()]
print(f"\nAverage neighbors found per user: {np.mean(neighbor_counts):.1f}")
print(f"Total Users processed: {len(top_20_neighbors)}")

⏳ Loading and Preprocessing...
✅ Data Ready: (35638, 4) | Users: 1000
🚀 Calculating Discounted Similarities (Beta=1.0)...

🔍 Extracting top 199 neighbors (20%) per user...

✅ Results (Discounted Distance-Based Similarity):
User 7799420340:
   -> Top Neighbors: ['B01929H4VM', 'B001GFKIBE', 'B00330UFQS', 'B00124BPBG', 'B00BWGHIHY']...
   -> Scores: [0.571 0.562 0.5   0.5   0.5  ]
User 9434682614:
   -> Top Neighbors: ['B009VLX8FS', 'B0092MKTL2', 'B008GVTSL2', 'B0092MKTWQ', 'B005MVLI8A']...
   -> Scores: [0.4  0.25 0.25 0.25 0.25]
User 9714721180:
   -> Top Neighbors: ['B005MVLI8A', 'B015U2LLIW', 'B00136NUG6', 'B01929H4VM', 'B001GFKIBE']...
   -> Scores: [0.732 0.65  0.5   0.417 0.4  ]
User B000JF21D0:
   -> Top Neighbors: ['B00TJ6S4RW', 'B0043ZDFEQ', 'B000W11N6W', 'B001GH1ZQE', 'B004EI3ON4']...
   -> Scores: [0.417 0.417 0.417 0.4   0.4  ]
User B000QOEN6W:
   -> Top Neighbors: ['B000VZV9KY', 'B00137QS28', 'B000VWMTHE', 'B00136NFOI', 'B00GLP4DMO']...
   -> Scores: [0.5 0.5 0.5 0.5 0.4]

A

In [68]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1. Setup Data for Prediction
# ---------------------------------------------------------
# Calculate User Means (needed for the formula: r_v - mean_v)
# We treat 0 as missing, so we replace 0 with NaN to get the true mean
user_means = user_item_matrix.replace(0, np.nan).mean(axis=1)

# Create a "Centered" matrix (Rating - User Mean)
# This allows us to quickly grab (r_{v,i} - mean_v) during the loop
ratings_diff = user_item_matrix.replace(0, np.nan).sub(user_means, axis=0)

# ---------------------------------------------------------
# 2. Prediction Function
# ---------------------------------------------------------
def predict_second_round(target_users, neighbors_dict, similarity_df, ratings_diff_matrix, user_means):
    predictions = []
    
    print(f"🚀 Starting Prediction for {len(target_users)} users using Discounted Similarities...")
    
    for i, user in enumerate(target_users):
        # Progress indicator
        if i % 100 == 0 and i > 0:
            print(f"   ...Processed {i} users")
            
        # 1. Get the specific Top 20% neighbors for this user
        neighbors = neighbors_dict.get(user, [])
        
        if not neighbors:
            continue
            
        # 2. Identify items to predict
        # We only predict items that the neighbors have rated but the user hasn't
        # Getting neighbors' ratings (centered)
        neighbor_ratings = ratings_diff_matrix.loc[neighbors]
        
        # Identify items where at least one neighbor has a rating (not NaN)
        # and the target user has NOT rated (value is NaN in ratings_diff_matrix or 0 in original)
        candidates = neighbor_ratings.count() > 0 # Items neighbors rated
        user_rated = ~ratings_diff_matrix.loc[user].isna() # Items user rated
        items_to_predict = candidates.index[candidates & ~user_rated]
        
        # If too many items, let's limit to top 50 most popular among neighbors to save time
        if len(items_to_predict) > 50:
            items_to_predict = items_to_predict[:50]
            
        # 3. Prediction Loop per Item
        for item in items_to_predict:
            # Get neighbors who actually rated this specific item
            # (Slice the centered ratings for these neighbors on this item)
            item_ratings = neighbor_ratings[item].dropna()
            valid_raters = item_ratings.index
            
            if len(valid_raters) == 0:
                continue
                
            # Get Similarities (DS scores) for these specific raters
            # These are the "Discounted" weights
            sim_scores = similarity_df.loc[user, valid_raters]
            
            # --- THE FORMULA ---
            # Numerator: Sum( Sim * (NeighborRating - NeighborMean) )
            numerator = np.dot(sim_scores, item_ratings)
            
            # Denominator: Sum( |Sim| )
            denominator = sim_scores.abs().sum()
            
            if denominator == 0:
                continue
                
            # Calculate offset
            pred_offset = numerator / denominator
            
            # Final Prediction = UserMean + Offset
            final_pred = user_means[user] + pred_offset
            
            # Clip between 1 and 5
            final_pred = min(5.0, max(1.0, final_pred))
            
            predictions.append({
                'User': user,
                'Item': item,
                'Predicted_Rating': round(final_pred, 2),
                'Contributing_Neighbors': len(valid_raters)
            })
            
    return pd.DataFrame(predictions)

# ---------------------------------------------------------
# 3. Run the Prediction
# ---------------------------------------------------------
# We will predict for all users in our filtered set
all_users = user_item_matrix.index

# Run!
pred_df = predict_second_round(
    target_users=all_users,
    neighbors_dict=top_20_neighbors, # Dictionary from previous step
    similarity_df=sim_df,            # Matrix from previous step
    ratings_diff_matrix=ratings_diff,
    user_means=user_means
)

# ---------------------------------------------------------
# 4. Display & Analyze Results
# ---------------------------------------------------------
if not pred_df.empty:
    print("\n✅ Prediction Complete!")
    print(f"Generated {len(pred_df)} predictions.")
    
    # Sort by highest predicted rating to see recommendations
    top_recs = pred_df.sort_values(by=['User', 'Predicted_Rating'], ascending=[True, False])
    
    print("\n🔹 Sample Recommendations (User-Based CF with Discounted Similarity):")
    print(top_recs.head(10))
    
    # Optional: Save to CSV
    # top_recs.to_csv("Discounted_Predictions.csv", index=False)
else:
    print("❌ No predictions generated. Check if neighbors overlap on unrated items.")

🚀 Starting Prediction for 1000 users using Discounted Similarities...
   ...Processed 100 users
   ...Processed 200 users
   ...Processed 300 users
   ...Processed 400 users
   ...Processed 500 users
   ...Processed 600 users
   ...Processed 700 users
   ...Processed 800 users
   ...Processed 900 users

✅ Prediction Complete!
Generated 49983 predictions.

🔹 Sample Recommendations (User-Based CF with Discounted Similarity):
          User                  Item  Predicted_Rating  Contributing_Neighbors
0   7799420340  A0072041HVZ3465DXUOR              5.00                       6
2   7799420340        A101PYPZ8ASKCV              5.00                       6
5   7799420340        A1063L1ALPA9WZ              5.00                       1
7   7799420340        A107OPFJV11RKO              5.00                       2
9   7799420340        A10D50UBEF3NUF              5.00                       1
10  7799420340        A10DYTGIWV3RJZ              5.00                       3
11  7799420340      

In [69]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ---------------------------------------------------------
# 1. Setup Data (Assume 'user_item_matrix' exists from previous steps)
# ---------------------------------------------------------
# Ensure we are using the exact same matrix for fair comparison
# If user_item_matrix is not in memory, please run the loading block from previous steps.

# ---------------------------------------------------------
# 2. Generate List A: Standard Pearson/Cosine (Step 2 Approach)
# ---------------------------------------------------------
print("🔄 Generating List A (Standard Cosine)...")
# Mean centering
user_means = user_item_matrix.replace(0, np.nan).mean(axis=1)
matrix_centered = user_item_matrix.sub(user_means, axis=0).fillna(0)

# Standard Cosine
sim_matrix_standard = cosine_similarity(matrix_centered)
sim_df_step2 = pd.DataFrame(sim_matrix_standard, index=user_item_matrix.index, columns=user_item_matrix.index)

# Get Top 20% for Step 2
neighbors_step2 = {}
k_percent = 0.20
for user in sim_df_step2.index:
    # Sort descending, drop self
    candidates = sim_df_step2.loc[user].drop(user).sort_values(ascending=False)
    k = max(1, int(len(candidates) * k_percent))
    neighbors_step2[user] = candidates.head(k).index.tolist()

# ---------------------------------------------------------
# 3. Generate List B: Discounted DS (Step 5 Approach)
# ---------------------------------------------------------
# Assuming 'sim_df' from the previous Discounted calculation is available.
# If not, we use the logic from the previous prompt (sim_df in code block above).
# For this comparison script, let's assume `top_20_neighbors` (from Step 5) is ready.
neighbors_step5 = top_20_neighbors # From previous code block

# ---------------------------------------------------------
# 4. Compare the Lists
# ---------------------------------------------------------
comparison_results = []

print("\n⚔️  Comparing Neighborhoods (Step 2 vs Step 5)...\n")

users_to_check = list(neighbors_step5.keys())[:10] # Check first 10 users

for user in users_to_check:
    list_a = set(neighbors_step2.get(user, []))
    list_b = set(neighbors_step5.get(user, []))
    
    # Calculate Jaccard Similarity (Intersection over Union)
    intersection = list_a.intersection(list_b)
    union = list_a.union(list_b)
    
    jaccard_score = len(intersection) / len(union) if len(union) > 0 else 0
    
    comparison_results.append({
        'User': user,
        'Size_Step2': len(list_a),
        'Size_Step5': len(list_b),
        'Overlap_Count': len(intersection),
        'Jaccard_Sim': round(jaccard_score, 3)
    })

# Create DataFrame
comp_df = pd.DataFrame(comparison_results)

print(comp_df)

print("\n-------------------------------------------------------------")
print(f"Average Jaccard Overlap: {comp_df['Jaccard_Sim'].mean():.3f}")
print("-------------------------------------------------------------")

🔄 Generating List A (Standard Cosine)...

⚔️  Comparing Neighborhoods (Step 2 vs Step 5)...

         User  Size_Step2  Size_Step5  Overlap_Count  Jaccard_Sim
0  7799420340         199         199            162         0.69
1  9434682614         199          13              9         0.04
2  9714721180         199          73             53         0.24
3  B000JF21D0         199          73             57         0.27
4  B000QOEN6W         199         199            169         0.74
5  B000QP4IBG         199         132            115         0.53
6  B000S3BNY6         199         199            173         0.77
7  B000S51XMQ         199          93             61         0.26
8  B000S56380         199         199            179         0.82
9  B000SXM2AK         199         199            155         0.64

-------------------------------------------------------------
Average Jaccard Overlap: 0.499
-------------------------------------------------------------
